# JPEG compression with optimised Huffman tables
This tutorial builds on the [JPEG compression format one](./jpeg-baseline.ipynb) and explores the use of custom Huffman tables to improve the coding efficiency. We will comment the main two phases of statistics collection and Huffman's tables design to finish off with an assessment of the coding efficiency improvement. The tutorial will follow the procedure outlined in the JPEG compression standard specification ([ITU-T T.81](https://www.itu.int/rec/T-REC-T.81)) to derive the Huffman tables. As usual, it is assumed that the reader is familiar with the well-known Huffman coding technique. Suggested readings to brush the concepts off or learn them are:
  * W. B Pennebaker and J. L. Mitchell, "JPEG: Still image data compression standard", Kluwer Academic Publishers, 638 pages, 1992.
  * David S. Taubman and Micheal W. Marcellin, "JPEG 2000: Image compression fundamentals, standards and practice", Kluwer Academic Press, 773 pages, 2002.

## Preliminary remarks
If some additional computational complexity can be afforded at the transmitter side, then the coding efficiency of an encoder complaint with the JPEG compression standard can be improved by deriving the Huffman tables on the data being encoded. The specification document of the JPEG compression standard provides a set of Huffman tables, the so-called *default tables*, which can be used in case either the encoder's complexity is limited (i.e. no ad-hoc Huffman tables can be derived) or the encoder's does not have a clue on which content and coding rates will be dealing with. However, as mentioned in this [tutorial](./jpeg-baseline.ipynb), such default tables have been derived over content whose resolution and coding rates might not be representative of nowadays' application scenarios. As an example, modern images shared through the Internet have at least 1080p (1920 $\times $ 1080) resolution. Moreover, screenshots of a computer's desktop are very common as material compressed with JPEG as opposed to the content used during the definition of the standard which was more camera captured. Notwithstanding that the JPEG compression standard is not the best format to encode text (JPEG 2000 or the H.265/HEVC screen content coding profile are better suited for this purpose), it is however reasonable to expect that the statistics of run length pairs and/or DC predicted residuals will significantly be different from their camera captured counterpart. To implement the derivation, use and transmission of custom Huffman tables, we will modify the `jpeg_encoding` method from the `encoder.py` Python package to extend the encoder's workflow with the following two processing stages:
 * Collection of the symbols statistics
 * Design of Huffman tables compliant with the JPEG compression standard's specification

After the ad-hoc HUffman tables have been derived, the entropy coding phase specified by the JPEG compression standard can proceed as detailed [ealier](./jpeg-baseline.ipynb). The following subsections will elaborate on these two phases.

## Collection of the symbols statistics
When performing quantisation over an 8 $\times$ 8 block of coefficients, the encoder can count the frequency of each symbol associated with the different quantities passed through the entropy coding phase. As we know already, these quantities are:
 * Prediction residuals associated with the DC coefficients of both luma and chroma (i.e. Cb and Cr) components
 * Run length pairs re-arranged in the extended alphabet symbols associated with the AC coefficients of both luma and chroma components

To accomplish this task, an additional loop over all 8 $\times$ 8 blocks has been added to the workflow of `jpeg_encoding` straight after the forward DCT and uniform quantisation have been performed. The method `get_block_symbols` from the `entropy.py` Python package, mimics the processing performed during entropy coding to generate the DC predicted residuals and run length pairs. All the symbols are then inserted in a Python list so that their frequency can be calculated afterwards. There are four of such lists: one for the DC prediction residuals associated with the luma component, one for the DC prediction residuals associated with the Cb and Cr components, one for the run length pairs associated with AC coefficients of luma and finally one list for the run length pairs associated with the AC coefficients of Cb and Cr components.

## Derivation of the Huffman tables
Once all the four lists above have been populated for all 8 $\times$ 8 blocks of coefficients, the Huffman tables can be generated by this sequence of ordered steps:
 1. Calculation of the frequency of each symbol
 1. Application of the Huffman coding procedure to derive the `bits` and `values` 1D arrays definining a Huffman table (see the [previous tutorial](./jpeg-baseline.ipynb) for more details)
 1. Limitation of the length of codewords to 16 bits

The above steps are applied four times, one for each list of statistics collected before. We note that the first step could have been performed during the collect of statistics: we know that there will be eleven categories for the residuals DC symbols and at most 162 run length pair symbols. Accordingly our four previous lists could have had these sizes, been initialised to zero and then each entry would have been increment accordingly. Moreover, the last step is only required by the JPEG compression standard given that it mandates that all codewords have maximum lenght of 16 bits to allow speculative decoding techniques to be devised at the receiver's side.

### Symbol frequency calculation and table derivation
As mentioned already, the Huffman coding procedure is well-known and widely used by the data coding community. As such, there are plenty of off-the-shelf libraries which compute the optimal codewords according to the procedure defined by David Huffman. As such, one could use the Python package [huffman](https://pypi.org/project/huffman/) to derive the table for each list of statistics. However, the JPEG compression format constraints the symbols not to have an all 1s codeword. Such a constraint it is required since the all 1s codeword is reserved as prefix for the so-called extension codes used in the progressive and hierarchical modes of operation of the JPEG compression standard (not covered in this tutorial). Section K.2 of the JPEG compression standard specification ([ITU-T T.81](https://www.itu.int/rec/T-REC-T.81)), reports a procedure which resembles the canonical Huffman's algorithm. The procedure uses three 1D arrays with 257 entries each, denoted as follows:
 * **FREQ**: Stores the occurence of the $i$-th symbol. Zero value entries are associated with those symbols which are not present in the content being encoded. We note that 257 entries is more than needed for 8 bits per pixel content. Indeed the number is defined to accomodate higher bits per pixel images
 * **CODESIZE**: Stores the length of the codeword associated with the $i$-th symbol
 * **OTHERS**: Stores the index to the next symbol in the chain of symbols encoded with a given length of codeword

The procedure then loops over all elements in the array **FREQ** to find two symbols: one with the lowest frequency (denoted as $c_1$) and the other with the second lowest frequency (denoted as $c_2$). These two symbols are merged together at the position associated with $c_1$ by adding the frequency of $c_2$, incrementing the **CODESIZE** entry for $c_1$ and all its symbols dependent on $c_1$ (as indicated by the array **OTHERS**). The same operations are also applied to the symbol associated with $c_2$ with the fundamental different that its entry in the **FREQ** array is set to zero (this is because its frequency value has previouly been merged with $c_1$'s).

To avoid that any symbol will end up having an all 1s codeword, the procedure described in Section K.2 of the JPEG compression standard specification artificially sets the entry for symbol 256 to one: this symbol is illegal for the standard since there could be only 256 symbols whose values lie in the range $[0, 255]$ inclusive. By setting the frequency of symbol 256 to one, i.e. the minimum possible, the procedure guarantees that only this symbol will be associated with an all 1s codeword by the Huffman's pairing algorithm. The reader might wonder what happens if one of the symbols with a legal value is assigned to frequency equal to one. The specification document mandates to find the symbol with the lowest frequency value and among two or more with such a minimum value the one associated with the largest symbol's value is selected, that is 256. We note that the same results could have been obtained by invoking the `codebook` method from the Python package [huffman](https://pypi.org/project/huffman/) where the input consists of symbol value and symbol frequency pairs. Still to avoid that a given symbol is assigned with the all 1s codeword, the pair $(256, 1)$ must be added to the input presented to the `codebook` method. We preferred however to implement from scratch the Huffman table derivation procedure by reading the specification document so that the reader can have an example of translation of the flowcharts contained in that document into Python code.

### Limitation of the codeword length to 16 bits
From the **CODESIZE** array the `bits` can be derived. All entries in the **CODESIZE** array are scanned and, those which are different from zero, are used as index to increment by one the corresponding entry in the `bits` array (which is initialised to zero). At this point of the processing there is no guarantee that codewords with length greater than 16 bits don't exist. Due to the constraint mandated by the JPEG specification document, one needs to reshuffle the entries in the `bits` array. The procedure depicted in Figure K.3 of the specification document presents a method to limit the codeword length to 16 bits. Essentially, the algorithm starts from the largest codeword length and removes a pair of symbols to assigns their prefix (which has one bit less) to one symbol of the pair whilst the other will get the prefix of the next shortest entry in `bits` whose value is not zero. The process continues until there are no more symbols whose codeword's length is greater than 16 bits.

### Putting all together
Remembering that the spirit of this tutorial is also to implement what it is described in this notebook, we shall briefly describe which Python methods and packages have been devised to derive the custom Huffman tables.

As mentioned above, the symbols' statistics are collected after quantisation takes place in the `jpeg_encoding` workflow. At this point the method `design_huffman_table` from the `entropy.py` Python package is invoked with a list of integers as its input, corresponding to all values collected over residuals DC and AC values for all colour components. The method then calls the following three sub methods to carry out the processing described in the following itemised list:
  * `derive_huffman_table`: Implements the derivation of the Huffman tables as per the procedure described in Section K.2 of the specification document. The output of this method are the arrays `bits` and **CODESIZE**. The former is used as input to the following sub method whilst the latter is provided as input to the third sub method
  * `limit_codewords_length`: Implements the limitation of the codewords to 16 bits as described in Section K.2 and Figure K.3 of the specification document. The input to this method is the `bits` array and the output is the modified `bits` array where also the codeword associated with symbols 256 is removed
  * `sort_input`: Derives the `values` array from the **CODESIZE** one (see below for more details).

## Entropy encoding with custom Huffman tables
After the `bits` array has been adjusted so that there are no codewords whose length is larger than 16 bits, the `values` array can be derived. Accordingly, each possible value for the **CODESIZE** entries (i.e. 1 to 32) is considered and, for each one of them, the position (index) in the **CODESIZE** array is appended to `values`. Such position corresponds to the symbol values which have been paired by the Huffman algorithm with a codeword with a given number of bits.

After the `values` array is also available, the Huffman table in the format expected by the `jpeg_encoding` workflow is generated for each of the four class of symbols where the custom tables have been generated. The entropy encoding procedure then continues as already described in the [previous tutorial](./jpeg-baseline.ipynb).

## Coding efficiency assessment
It is now time to measure how much coding efficiency improvement the derivation of the Huffman tables can bring to a compliant JPEG compression standard encoder. The beauty of working on the entropy coding module implies that we can just measure the coding rate reduction associated with the encoder using the custom tables. As mentioned at the beginning of this tutorial, we note that the default Huffman tables of the JPEG compression standard have been derived over camera captured content whose resolution was quite different with respect to the current computers' monitors. Moreover, we already hinted that over screen content images, the derivation of custom tables should bring more benefits than plain use of the default ones. Accordingly, we will assess the coding efficiency improvement over two images: one is the canonical `peppers` with resolution 512 $\times$ 512 whilst the other is crop of the `wikipedia` image used in this [tutorial](../../entropy-coding/golomb-coding/golomb-coding-tutorial.ipynb) cropped to the top left quadrant (i.e. with resolution 480 $\times$ 270).

The following Python code cell performs JPEG encoding with the `jpeg_encoding` method from two different Python packages: one is the `encoder.py` package used in this [initial tutorial](./jpeg-baseline.ipynb) whilst the other is from the `encoder_ht_optimised.py`, implemented for the current tutorial. Both the Peak-Signal-to-Noise-Ratio (PSNR) over the luma component and the coding rate are measured for both encoders and over a range of values for the quality factor.

In [ ]:
import numpy as np
from encoder import jpeg_encoding
from encoder_ht_optimised import jpeg_encoding_ht_opt
import cv2
from ct import rgb_to_ycbcr_bt601
import os

images = ["../../input-data/rex-wikipedia-cropped.png", "../../input-data/peppers.tiff"]
qualities = [1, 20, 40, 60, 80, 90, 100]
bitstream_name = "bitstream.jpg"
psnr_default, psnr_opt = np.zeros((len(images), len(qualities))), np.zeros((len(images), len(qualities)))
bpp_default, bpp_opt = np.zeros((len(images), len(qualities))), np.zeros((len(images), len(qualities)))

for image_idx, im in enumerate(images):
    I = cv2.imread(im, cv2.IMREAD_UNCHANGED)
    image_ref_ycbcr = rgb_to_ycbcr_bt601(I[:, :, 0], I[:, :, 1], I[:, :, 2])
    rows, cols = I.shape[0], I.shape[1]
    
    for quality_idx, q in enumerate(qualities):
        # Default Huffman tables
        bytes_total, _ = jpeg_encoding(image_ref_ycbcr, bitstream_name, q)
        image_rec = cv2.imread(bitstream_name, cv2.IMREAD_UNCHANGED).astype(np.uint8)
        image_rec_ycbcr = rgb_to_ycbcr_bt601(image_rec[:, :, 2], image_rec[:, :, 1], image_rec[:, :, 0])
        psnr_default[image_idx, quality_idx] = 10 * np.log10(255**2 / np.mean(np.square(image_ref_ycbcr[:, :, 0] - image_rec_ycbcr[:, :, 0])))
        bpp_default[image_idx, quality_idx] = bytes_total * 8 / rows / cols

        # Optimised Huffman tables
        bytes_total, _ = jpeg_encoding_ht_opt(image_ref_ycbcr, bitstream_name, q)
        image_rec = cv2.imread(bitstream_name, cv2.IMREAD_UNCHANGED).astype(np.uint8)
        image_rec_ycbcr = rgb_to_ycbcr_bt601(image_rec[:, :, 2], image_rec[:, :, 1], image_rec[:, :, 0])
        psnr_opt[image_idx, quality_idx] = 10 * np.log10(255**2 / np.mean(np.square(image_ref_ycbcr[:, :, 0] - image_rec_ycbcr[:, :, 0])))
        bpp_opt[image_idx, quality_idx] = bytes_total * 8 / rows / cols

os.remove(bitstream_name)

The data measured are then displayed in the following Python code cell where also the average coding rate reduction with respect to the encoder which uses the default Huffman table is displayed.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 15))
titles = ["Wikipedia", "Peppers"]

for image_idx in range(len(images)):
    ax = plt.subplot(2, 1, image_idx + 1)
    cr_improvement_avg = np.average(np.divide(bpp_default[image_idx, :] - bpp_opt[image_idx, :], bpp_default[image_idx, :]) * 100)
    plt.plot(bpp_default[image_idx, :], psnr_default[image_idx, :], "b-o", linewidth=3, markersize=10, label="Default tables")
    plt.plot(bpp_opt[image_idx, :], psnr_opt[image_idx, :], "r-*", linewidth=3, markersize=10, label="Optimised tables")
    plt.grid()
    plt.legend(prop={'size': 16})
    plt.xlabel("Coding rate [bpp]", fontsize=16)
    plt.ylabel("PSNR-Y [dB]", fontsize=16)
    final_title = f"{titles[image_idx]} - Average gain: {cr_improvement_avg:.2f}%"
    ax.set_title(final_title, fontsize=20)

## Discussion and conclusions
Over the two images tested it was assessed that the average coding efficiency could be improved by up to 17.6\% and at least by 13.7\%. The results obtained confirm the initial expectation whereby the use of custom tables is more beneficial for screen content where the statistics of run length pairs are more different than their camera captured counterpart. For both images we note a more pronounced coding efficiency reduction at very high and very low coding rates. In the former case the category codes for AC and DC residuals are distributed differently when the coding rate is below 4 bits per pixel. In the latter instead, there will be more run length pairs with particular average values, hence a custom Huffman table is more efficient. All these considerations are of course related to the coding rate's reduction. If the encoder's complexity has to be taken into account, then the need to wait for the entire image to be processed (i.e. to wait for the symbols' statistics to be available) may be limiting in some application scenarios.

To conclude we hope the reader has found this tutorial useful, most notably to unveil some implementation details of the JPEG compression format (e.g. how the all 1s codeword is avoided). Moreover, we hope to have provided a ballpark figure on what one should expect when custom Huffman tables are used in lieu of the default ones.